# CHORDictor

## Description

### Project Scope
A Python program that can automatically annotate the jazz chord progressions for a MIDI file. For any ambiguous chords or sections which the program cannot determine an annotation for, the program will flag them and the human user can manually enter the corresponding label to ensure the annotation accuracy.

### Data
* Jazz Piano MIDI Multitracks
* Size: 310 MIDI files
* Source: https://bushgrafts.com/midi/

## Member Contributions

### Brandon Carone
* Initial scope of the project
* MIDI data loading and parsing
* MIDI Chord Templates - 33 chords in 12 keys
* Mapping notes to chords (templates)


### Kenny Huang
* MIDI data loading, parsing, & visualizing
* Selected method of parsing MIDI file (mido over music21)
* Converted key format from music21 to librosa
* Converted accidentals formatting
* Obtained music21's in-build chord-detection algorithm
* Helped write some MIDI processing functions

### Richa Namballa
* Documented code
* Wrote helper functions
* Quantized MIDI file
* Time-aligned MIDI parsing
* Converted MIDI to note names
* Extracted timestamps of each note
* Separated notes into individual beats

## Install & Import Libraries

In [ ]:
!pip install mido
!pip install music21

In [ ]:
import os
import copy
import numpy as np
import pandas as pd
import librosa
import mido
import re
import matplotlib.pyplot as plt
from glob import glob
from music21 import midi, chord
from tqdm import tqdm

## Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# upload the jazz_midi folder to mir_datasets folder in Drive
midi_dir = '/content/drive/MyDrive/mir_datasets/jazz_midi/*.mid'
midi_files = []
for midi_file in tqdm(glob(midi_dir)):
    midi_files.append(midi_file)
midi_files.sort()

## Data Preprocessing

### Helper Functions

In [ ]:
def quantize_file(midi_file_path, out_path, divisor=8, offsets=True, durations=True):
    """
    Quantize a the inputted MIDI file.
    
    :param midi_file_path: (str) path to MIDI file to be quantized
    :param out_path: (str) path to written quantized MIDI file
    :param divisor: (int) the subdivision of a quarter note to quantize to,
                    the default value is 8, which corresponds to a 1/32nd note
    :param offets: (bool) whether to quantize the note offsets
    :param durations: (bool) where to quantize the note durations
    """
    mf = midi.MidiFile()
    mf.open(midi_file_path, 'rb')
    mf.read()
    mf.close()
    
    s = midi.translate.midiFileToStream(mf)
    s.quantize((divisor,),
               processOffsets=offsets,
               processDurations=durations,
               inPlace=True, recurse=True)
    
    mf_quant = midi.translate.streamToMidiFile(s)
    
    mf_quant.open(out_path, 'wb')
    mf_quant.write()
    mf_quant.close()

In [ ]:
def load_midi(midi_file_path, quantize=True):
    """
    Loads MIDI file using mido.
    
    :param midi_file_path: (str) path to MIDI file
    :param quantize: (bool) whether to quantize the MIDI file
    :return: (MidiFile) mido MIDI file object
    """
    if quantize:
        # create a temporary MIDI file with quantization
        quantize_file(midi_file_path, 'temp.mid')
        # load
        mid = mido.MidiFile('temp.mid')
        print(f'Temporary MIDI file created at {os.path.abspath("temp.mid")}')
        # remove temporary file
        os.remove('temp.mid')
        if os.path.exists('temp.mid'):
          print('Unable to remove temporary MIDI file!')
    else:
        # load
        mid = mido.MidiFile(midi_file_path)
    return mid

In [ ]:
def get_tempo(mid):
    """
    Determine tempo of the MIDI file.
    
    :param mid: (MidiFile) mido MIDI file object
    :return: (int) tempo in microseconds per beat
    """
    tempo_list = []
    for track in mid.tracks:
        for msg in track:
            if msg.type == 'set_tempo':
                 tempo_list.append(msg.tempo)

    if len(tempo_list) > 0:
        # return the first tempo
        return tempo_list[0]
    else:
        # default tempo from mido
        return 500000

In [ ]:
def get_bpm(mid):
    """
    Determine the BPM of the MIDI file
    
    :param mid: (MidiFile) mido MIDI file object
    :return: (int) BPM
    """
    # compute tempo
    tempo = get_tempo(mid)
    # convert tempo to BPM and round to the nearest integer
    bpm = round(mido.tempo2bpm(tempo))
    return bpm

In [ ]:
def get_metadata(mid):
    """
    Print metadata from MIDI file
    
    :param mid: (MidiFile) mido MIDI file object
    """
    for track in mid.tracks:
        for msg in track:
            if msg.is_meta:
                print(msg)
    else:
        pass

In [ ]:
def remove_octave(note):
    """
    Strips the octave number from a note.
    
    :param note: (str) note in string form (e.g. 'C5')
    :return: (str) note without octave (e.g. 'C')
    """
    # remove digits from the string
    chars = [i for i in note if not i.isdigit()]
    # rejoin characters
    note_without_octave = ''.join(chars)
    return note_without_octave

In [ ]:
def calculate_similarity(list1, list2):
    """
    Computes the proportion of overlap between two lists.

    :param list1: (list)
    :param list2: (list)
    :return: (float) similarity
    """
    denom = max(len(list1), len(list2))  # use the length of the longer list as the denominator
    intersect = list(set(list1) & set(list2))  # find the intersection of two sets
    numer = len(intersect)
    if denom > 0:
        sim = numer / denom
    else:
        # avoids ZeroDivisionError
        sim = 0
    return sim

### Using `mido` to Read MIDI Files

In [ ]:
# adapted from: https://stackoverflow.com/questions/63105201/python-mido-how-to-get-note-starttime-stoptime-track-in-a-list

# midi file to array
def midi_parser(mid):
    """
    Converts a mido MIDI file into an array of note on/off with timestamps

    :param mid: (MidiFile) MIDI file object loaded with mido
    :return: (np.ndarray) array of [type, note, time, channel]
    """
    midi_array = []
    
    # get the tempo in microseconds per beat
    tempo = get_tempo(mid)
    
    # find how many ticks are in a single beat
    tpb = mid.ticks_per_beat

    # iterate through each track individually
    for track in mid.tracks:
        track_info = {}
        metadata   = {}  # track metadata
        note_array = []  # track specific note information
        midi_dict  = []  # a list of dictionaries
        
        # iterate through each message in the track
        for msg in track:
            # TODO: check whether there are other types of messages
            # if (msg.type == 'note_on' or msg.type == 'note_off' or msg.type == 'control_change'):
            if not msg.is_meta:
                # put all note on/off in midinote as dictionary
                midi_dict.append(msg.dict())
            elif msg.type == 'track_name':
                # save the track name as metadata
                metadata['name'] = msg.name
            elif msg.type == 'time_signature':
                # save the time signature as metadata
                metadata['time_signature'] = f'{msg.numerator}/{msg.denominator}'
            elif msg.type == 'key_signature':
                # save the key as metadata
                metadata['key'] = msg.key
            else:
                pass
        
        # add the metadata information
        track_info['metadata'] = metadata

        # change time values from delta to relative time
        event_start = 0
        for d in midi_dict:
            time_in_s = mido.tick2second(d['time'], tpb, tempo)
            time = time_in_s + event_start
            d['time'] = time
            event_start += time_in_s

            # make every note_on with 0 velocity note_off
            if d['type'] == 'note_on' and d['velocity'] == 0:
                d['type'] = 'note_off'

            # put note, starttime, stoptime, as nested list in a list
            # format is [type, note, time, channel]
            event = []
            if d['type'] == 'note_on' or d['type'] == 'note_off':
                # do not include control change values
                event.append(d['type'])
                event.append(d['note'])
                event.append(d['time'])
                event.append(d['channel'])
                note_array.append(event)
        
        # add the note information
        track_info['note_data'] = note_array
        
        # add track to entire midi file array
        midi_array.append(track_info)

    return midi_array

In [ ]:
# parsing an example midi file
mid_file   = midi_files[1]  # example file: "A Sleepin' Bee.mid"
mid        = load_midi(mid_file, quantize=True)
midi_array = midi_parser(mid)

In [ ]:
# assigning plot vectors
note_on_values  = [row[1] for row in midi_array[0]['note_data'] if row[0] == 'note_on']
note_on_times   = [row[2] for row in midi_array[0]['note_data'] if row[0] == 'note_on']
note_off_values = [row[1] for row in midi_array[0]['note_data'] if row[0] == 'note_off']
note_off_times  = [row[2] for row in midi_array[0]['note_data'] if row[0] == 'note_off']

# plotting midi
fig, ax = plt.subplots(figsize=(10, 5))
ax.scatter(note_on_times, note_on_values, s=3.5, color='green')
ax.scatter(note_off_times, note_off_values, s=3.5, color='red')
ax.set(title=f'MIDI Notes for {mid_file}')
ax.set(ylabel='MIDI Value (0-127)')
ax.set(xlabel='Time (sec)')
ax.grid(visible='on', which='both')
fig.tight_layout()
fig.show()

### Convert MIDI to Note Name

In [ ]:
def convert_key_format(midi_array):
    """
    Convert key from mido format to librosa format.
    
    :param midi_array: (list) parsed MIDI array
    :return: (str) converted key in librosa format (e.g. C:maj)
    """
    # a total of 30 applicable music key signatures
    # A Am    A#m Ab Abm 
    # B Bm        Bb Bbm 
    # C Cm C# C#m Cb 
    # D Dm    D#m Db 
    # E Em        Eb Ebm 
    # F Fm F# F#m 
    # G Gm    G#m Gb 
    chord_keys = {'A':'A:maj', 'Am':'A:min',                'A#m':'A#:min', 'Ab':'Ab:maj', 'Abm':'Ab:min',
                  'B':'B:maj', 'Bm':'B:min',                                'Bb':'Bb:maj', 'Bbm':'Bb:min',
                  'C':'C:maj', 'Cm':'C:min', 'C#':'C#:maj', 'C#m':'C#:min', 'Cb':'Cb:maj',
                  'D':'D:maj', 'Dm':'D:min',                'D#m':'D#:min', 'Db':'Db:maj',
                  'E':'E:maj', 'Em':'E:min',                                'Eb':'Eb:maj', 'Ebm':'Eb:min',
                  'F':'F:maj', 'Fm':'F:min', 'F#':'F#:maj', 'F#m':'F#:min',
                  'G':'G:maj', 'Gm':'G:min',                'G#m':'G#:min', 'Gb':'Gb:maj'}
    
    converted_key_format = 'C:maj'  # default key
    for track in midi_array:
        if 'key' in track['metadata']:
            # find the corresponding librosa key format
            converted_key_format = chord_keys[track['metadata']['key']]
    return converted_key_format

In [ ]:
def convert_notes(arr, key=None):
    """
    Convert MIDI integers to note names.
    
    :param arr: (list) parsed MIDI array
    :param key: (str) predetermined key in librosa format
                if not provided, the key is extracted from the MIDI file
    :return: (list) MIDI array with note names in provided key
    """
    if key is None:
        midi_key = convert_key_format(arr)
    else:
        midi_key = key
    for track in arr:
        new_data = []
        if len(track['note_data']) > 0:
            for note in track['note_data']:
                # convert MIDI number to note name
                note[1] = librosa.midi_to_note(note[1], key=midi_key)
                new_data.append(note)
            track['note_data'] = new_data
    return arr

In [ ]:
note_array = convert_notes(midi_array)

### Chunk the MIDI Data into Beats

In [ ]:
def process_times(data):
    """
    Convert note data to find the onset and offset times for each note, along with its duration.
    
    :param data: (list) note data
    :return: (list) time data
    """
    # convert data to a pandas dataframe
    df = pd.DataFrame(data)
    # label the columns
    df.columns = ['event', 'note', 'time', 'channel']
    # sort the data by note first, then time to get each note on and off in consecutive rows
    df.sort_values(['note', 'time'], inplace=True)
    
    notes    = []
    note_on  = []
    note_off = []
    channel  = []
    
    # group every two rows together (one note on and one note off)
    for i, g in df.groupby(np.arange(len(df)) // 2):
        notes   .append(g.iloc[0, 1])
        note_on .append(g.iloc[0, 2])
        note_off.append(g.iloc[1, 2])
        channel .append(g.iloc[0, 3])

    # create a new dataframe
    df_times = pd.DataFrame({'note': notes, 'time_on': note_on, 'time_off': note_off, 'channel': channel})
    # sort by time to reconstruct the note sequence
    df_times.sort_values(['time_on', 'time_off'], ignore_index=True, inplace=True)
    # compute the duration of each note
    df_times['duration'] = df_times['time_off'] - df_times['time_on']
    # reorder columns
    df_times = df_times[['note', 'time_on', 'time_off', 'duration', 'channel']]
    # convert to list
    times_list = df_times.values.tolist()
    return times_list

In [ ]:
for d in note_array:
    if len(d['note_data']) > 0:
        d['time_data'] = process_times(d['note_data'])

In [ ]:
def get_notes_by_beat(data, seconds_per_beat, total_time, keep_embellishments=True, subdivision=4):
    """
    Get the notes which are played during each beat.
    
    :param data: (list) note data
    :param seconds_per_beat: (float) length of a beat in seconds
    :param total_time: (float) length of the MIDI file in seconds
    :param keep_embellishments: (bool) whether to keep embellishment tones
                                (e.g. passing or neighboring tones)
    :param subdivision: (int) if keep_embellishments is False, the minimum subdivision
                              of the beat for which a note can be included in the final output
                              (e.g. if subdivision=2 and the timesignature is 4/4, any note
                              less than an eighth note will be dropped)
    :return: (list) nested list where each item is a list of notes
                    played during beat i
    """
    # get the timestamps of each beat
    beat_timestamps = list(np.arange(0, total_time, spb))
    
    # create an empty nested list where each item represents one beat
    notes_by_beat = [[] for i in np.arange(len(beat_timestamps))]
    
    # convert list to dataframe
    df = pd.DataFrame(data, columns=['note', 'time_on', 'time_off', 'duration', 'channel'])
    
    if not keep_embellishments:
        df = df[df['duration'] >= round(seconds_per_beat / subdivision, 6)]
    
    # compute the number of beats in the duration
    df['num_beats'] = np.ceil(df['duration'] / seconds_per_beat)
    
    # iterate over each note
    for idx, row in df.iterrows():
        # find which beat the note starts on
        beat_start = sum([row['time_on'] > t for t in beat_timestamps]) - 1
        
        # add the note to the correct beats
        i = 0
        while i < row['num_beats']:
            notes_by_beat[beat_start + i].append(row['note'])
            i += 1
            
    # remove duplicates notes from a beat
    for i in range(len(notes_by_beat)):
        notes_by_beat[i] = list(set(notes_by_beat[i]))

    return notes_by_beat

In [ ]:
spb = 60 / get_bpm(mid)
for d in note_array:
    if 'time_data' in d.keys():
        d['beat_data'] = get_notes_by_beat(d['time_data'], spb, mid.length, keep_embellishments=False)

In [ ]:
note_array[0]

## Map Notes to Chords

### Using `music21`'s Magical In-built Chord Detection

In [ ]:
def convert_accidentals(chord_list):
    '''
    Convert accidentals to music21 format.

    :param chord_list: (list of list) chord list of list to be converted
    :return: (list) a list of converted accidentals
    '''
    converted_accidentals = [[note.replace("♭", "-").replace("♯", "#") for note in chord] for chord in chord_list]
    return converted_accidentals

In [ ]:
# returns chord type and root names
def chord_id(chord_list):
    '''
    music21 chord identifier function.

    :param chord_list: (list of list) chord list of list to be converted
    :return: (list) a list of chord types
    :return: (list) a list of root names
    '''
    chord_types = []    # list of chord types
    chord_roots = []    # list of chord roots
    for chord_idx in range(len(chord_list)):
        chord_type = chord.Chord(chord_list[chord_idx])   # gets the chord object
        chord_type.root(chord_type.bass())                # identifies the root of chord
        chord_types.append(chord_type.pitchedCommonName)  # identifies the chord type
        chord_roots.append(chord_type.root().name)        
    return chord_types, chord_roots

### Using our own MIDI Chord Templates

In [ ]:
# Run the necessary functions to create an array for our chord template
chords = '/content/drive/MyDrive/mir_datasets/all_chords.mid'     # Please upload the chords.midi file to the mir_datasets folder in Drive

chords_mid = load_midi(chords, quantize = False)

chord_array = convert_notes(midi_parser(chords_mid), key = note_array[0]['metadata']['key'] + ':maj')  #gets they key from the current song being analyzed

In [ ]:
for d in chord_array:
    if len(d['note_data']) > 0:
        d['time_data'] = process_times(d['note_data'])
        print(len(d['time_data']))

In [ ]:
spb

In [ ]:
chords_mid.length

In [ ]:
spb = 60 / get_bpm(chords_mid)
for d in chord_array:
    if 'time_data' in d.keys():
        d['beat_data'] = get_notes_by_beat(d['time_data'], spb, chords_mid.length, keep_embellishments=False)

In [ ]:
#Create a list including all of the chords in the chord template midi file
all_chords = []
for i in np.arange(1, len(chord_array)):
    all_chords = all_chords + [x for x in chord_array[i]['beat_data'][0::4] if x]

In [ ]:
#Remove all octave numbers from list
for i in np.arange(len(all_chords)):
    for j in np.arange(len(all_chords[i])):
        all_chords[i][j] = remove_octave(all_chords[i][j])

In [ ]:
len(all_chords)

In [ ]:
#Create lists of keys and chord names
keys = ['C', 'C#', 'Db', 'D', 'D#', 'Eb', 'E', 'F', 'F#', 'Gb', 'G', 'G#', 'Ab', 'A', 'A#', 'Bb', 'B', 'Cb']
chord_names = ['maj', 'maj7', '6', '6/9', 'maj9', 'maj9(no3)', 'maj9(no5)', '13', '13(no3)', '13(no5)', 'maj13', 'maj13(no11)', 'maj13(no5)', 'maj13(no9)', 'm', 'm7', 'm6', 'm6/9', 'm9', 'm9(no3)', 'm9(no5)', 'm11', 'm11(no3)', 'm11(no9)', 'm13',  'm13(no11)', 'm13(no5_no9_no11)',  'mmaj7', 'mmaj7/9', '7', '7sus', '9', '9sus', '9(no3)', '9(no5)', '11', '11(no3)', '11(no9)', 'dim', 'dim7', 'm7/-5', 'aug', 'aug7', '7/-5', '7/-9', '7/#9', '7#11', '5', 'add9', 'add11', 'sus4', 'sus2']

In [ ]:
# Create a dictionary with the chord labels as keys and a list of the notes in the chord as values
chord_dict = {}
counter = 0
for i in keys:
  for j in chord_names:
    chord_dict[i+j] = all_chords[counter]
    counter += 1

In [ ]:
chord_dict

In [ ]:
#Test a chord
chord_dict['Bm9(no5)']

In [ ]:
#Test similarity function
calculate_similarity(['B', 'E', 'G♭'], ['B', 'E', 'G♭', 'D'])

In [ ]:
#Get rid of the octaves in the note_array
for i in np.arange(len(note_array[0]['beat_data'])):
    for j in np.arange(len(note_array[0]['beat_data'][i])):
        note_array[0]['beat_data'][i][j] = remove_octave(note_array[0]['beat_data'][i][j])

In [ ]:
note_array[0]['beat_data']

In [ ]:
#Function that takes in the beat data and outputs the closest matches in the template

def chord_output(beat_data):
    song_chords = []
    for i in np.arange(len(note_array[0]['beat_data'])):
        similarity = []
        for j in np.arange(len(chord_dict)):
            #chords = list(chord_dict.values()) #or all_chords
            similarity.append(calculate_similarity(note_array[0]['beat_data'][i], all_chords[j]))
        song_chords.append(all_chords[np.argmax(similarity)])
    return song_chords

In [ ]:
chord_outputs = chord_output(note_array[0]['beat_data'])

In [ ]:
chord_outputs

In [ ]:
#Function that labels the chords

def label_chords(chords):
    labels = []
    for k in chords:
        labels.append([chord for chord, notes in chord_dict.items() if notes == k])
    return labels

In [ ]:
label_chords(chord_outputs)

# Scratch Work

### Using `music21` to Read MIDI Files (Disregard)

In [ ]:
test_midi = './test.mid'
test_midi0 = converter.parse(test_midi)
k = test_midi0.analyze('key')
print(k)
transposed = test_midi0.transpose(-12)  # do this with all the files if using music21

In [ ]:
test_list = []
for n in transposed.recurse().notes:
    print(n.offset, ' '.join(p.nameWithOctave for p in n.pitches))
    # test_list.append(a)
    # print(a)

In [ ]:
notes = ['D#3', 'C3', 'G2', 'G#1']

_notes = [re.sub('♯', '#', n) for n in notes]

_notes

c = chord.Chord(_notes)

c.commonName

c.root()

chord_template = load_midi('/Users/richa/Downloads/chords (every key).mid')

note_array = convert_notes(midi_parser(chord_template))

note_array

for d in note_array:
    if len(d['note_data']) > 0:
        d['time_data'] = process_times(d['note_data'])

spb = 60 / get_bpm(chord_template)
for d in note_array:
    if 'time_data' in d.keys():
        d['beat_data'] = get_notes_by_beat(d['time_data'], spb, mid.length)

note_array[0]['beat_data']